# Variable Definitions API demo

## Setup

In [ ]:
import os
from pprint import pprint
from typing import Any
from rich import print_json

import requests

token = os.environ.get("OIDC_TOKEN")
active_group = os.environ.get("DAPLA_GROUP_CONTEXT")

if not token:
    msg = (
        """Environment variable 'OIDC_TOKEN' not set, obtain the value for it by:
    1. Launch a service (Jupyterlab or VSCode) at https://lab.dapla.ssb.no
    2. Run command 'echo $OIDC_TOKEN'
    3. Run 'export OIDC_TOKEN=<value obtained from Dapla Lab>' on this machine.
"""
    )
    raise OSError(msg)

# Only available from Statistics Norway's networks, requires authorization
INTERNAL_SERVER = "https://metadata.intern.test.ssb.no"

auth = {"Authorization": f"Bearer {token}"}

def vardef_request(path: str, method: str="GET", body: dict | None = None) -> Any:
    """Make a request to vardef and return the json content from the response."""
    response = requests.request(method, f"{INTERNAL_SERVER}{path}?active_group={active_group}", headers=auth, json=body, timeout=10)
    response.raise_for_status()
    return response.json()

## List all variable definitions

In [ ]:
print_json(data=vardef_request("/variable-definitions"))

## Migrate a legacy definition from Vardok to Vardef

Note: This will raise an error if the short name already exists.

In [ ]:
# Uncomment to run, choose a variable definition which is not yet migrated.
# print_json(data=vardef_request("/vardok-migration/948", "POST"))

## Get one Variable Definition

In [ ]:
print_json(data=vardef_request("/variable-definitions/KiCPNU4n"))

## Get one Patch

In [ ]:
print_json(data=vardef_request("/variable-definitions/KiCPNU4n/patches/1"))

## Create a Patch

Only Published Variable Definitions may have Patches created on them, Draft Variable Definitions are updated in place.

Only the fields which are to be changed need to be supplied, the others are taken from the latest patch in the last Validity Period

In [ ]:
body = {
    "related_variable_definition_uris": [
        "https://example2.com/",
        "https://example.com/",
    ],
}

print_json(data=vardef_request("/variable-definitions/KiCPNU4n/patches", "POST", body))

## List Patches

In [ ]:
print_json(data=vardef_request("/variable-definitions/KiCPNU4n/patches"))